In [60]:
from dice import six_sided, four_sided, make_test_dice
from ucb import main, trace, interact

GOAL_SCORE = 100  # The goal of Hog is to score 100 points.


######################
# Phase 1: Simulator #
######################


def roll_dice(num_rolls, dice=six_sided):
    """Simulate rolling the DICE exactly NUM_ROLLS > 0 times. Return the sum of
    the outcomes unless any of the outcomes is 1. In that case, return 1.

    num_rolls:  The number of dice rolls that will be made.
    dice:       A function that simulates a single dice roll outcome.
    """
    # These assert statements ensure that num_rolls is a positive integer.
    assert type(num_rolls) == int, 'num_rolls must be an integer.'
    assert num_rolls > 0, 'Must roll at least once.'
    # BEGIN PROBLEM 1
    "*** YOUR CODE HERE ***"
    is_1 = False
    i, sum = 0, 0
    while i < num_rolls:
        x = dice()
        sum += x
        i += 1
        if x == 1:
            is_1 = True
    return sum if is_1 is False else 1
    # END PROBLEM 1


def digit_fn(digit):
    """Return the corresponding function for the given DIGIT.

    value:  The value which this function starts at.
    """
    # Error if DIGIT is not one of: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
    assert isinstance(digit, int) and 0 <= digit < 10
    # List of pre-defined functions
    f0 = lambda value: value + 1
    f1 = lambda value: value ** 2
    f2 = lambda value: value * 3
    f3 = lambda value: value // 4
    f4 = lambda value: value - 5
    f5 = lambda value: value % 6
    f6 = lambda value: int((value % 7) * 8)
    f7 = lambda value: int(value * 8.8)
    f8 = lambda value: int(value / 99 * 15) + 10
    f9 = lambda value: value
    # Mapping from digit to function
    if digit == 0:
        return f0
    elif digit == 1:
        return f1
    elif digit == 2:
        return f2
    elif digit == 3:
        return f3
    elif digit == 4:
        return f4
    elif digit == 5:
        return f5
    elif digit == 6:
        return f6
    elif digit == 7:
        return f7
    elif digit == 8:
        return f8
    elif digit == 9:
        return f9


def hefty_hogs(player_score, opponent_score):
    """Return the points scored by player due to Hefty Hogs.

    player_score:   The total score of the current player.
    opponent_score: The total score of the other player.
    """
    # BEGIN PROBLEM 2
    if opponent_score:
        while opponent_score:
            fn = opponent_score % 10
            player_score = digit_fn(fn)(player_score)
            opponent_score = opponent_score // 10
        return player_score % 30
    else:
        return 1
    # END PROBLEM 2


def take_turn(num_rolls, player_score, opponent_score, dice=six_sided, goal=GOAL_SCORE):
    """Simulate a turn rolling NUM_ROLLS dice,
    which may be 0 in the case of a player using Hefty Hogs.
    Return the points scored for the turn by the current player.

    num_rolls:       The number of dice rolls that will be made.
    player_score:    The total score of the current player.
    opponent_score:  The total score of the opponent.
    dice:            A function that simulates a single dice roll outcome.
    goal:            The goal score of the game.
    """
    # Leave these assert statements here; they help check for errors.
    assert type(num_rolls) == int, 'num_rolls must be an integer.'
    assert num_rolls >= 0, 'Cannot roll a negative number of dice in take_turn.'
    assert num_rolls <= 10, 'Cannot roll more than 10 dice.'
    assert max(player_score, opponent_score) < goal, 'The game should be over.'
    # BEGIN PROBLEM 3
    "*** YOUR CODE HERE ***"
    return roll_dice(num_rolls, dice) if num_rolls != 0 else hefty_hogs(player_score, opponent_score)
    # END PROBLEM 3


def hog_pile(player_score, opponent_score):
    """Return the points scored by player due to Hog Pile.

    player_score:   The total score of the current player.
    opponent_score: The total score of the other player.
    """
    # BEGIN PROBLEM 4
    "*** YOUR CODE HERE ***"
    player_one_digit, opponent_one_digit = player_score % 10, opponent_score % 10
    return player_one_digit if player_one_digit == opponent_one_digit else 0
    # END PROBLEM 4


def next_player(who):
    """Return the other player, for a player WHO numbered 0 or 1.

    >>> next_player(0)
    1
    >>> next_player(1)
    0
    """
    return 1 - who


def silence(score0, score1, leader=None):
    """Announce nothing (see Phase 2)."""
    return leader, None


def play(strategy0, strategy1, score0=0, score1=0, dice=six_sided,
         goal=GOAL_SCORE, say=silence):
    """Simulate a game and return the final scores of both players, with Player
    0's score first, and Player 1's score second.

    A strategy is a function that takes two total scores as arguments (the
    current player's score, and the opponent's score), and returns a number of
    dice that the current player will roll this turn.

    strategy0:  The strategy function for Player 0, who plays first.
    strategy1:  The strategy function for Player 1, who plays second.
    score0:     Starting score for Player 0
    score1:     Starting score for Player 1
    dice:       A function of zero arguments that simulates a dice roll.
    goal:       The game ends and someone wins when this score is reached.
    say:        The commentary function to call every turn.
    """
    who = 0  # Who is about to take a turn, 0 (first) or 1 (second)
    leader = None  # To be used in problem 7
    # BEGIN PROBLEM 5
    "*** YOUR CODE HERE ***"
    while score0 < goal and score1 < goal:
        if who:
             score1 += take_turn(strategy1(score1, score0), score1, score0, dice, goal)
             score1 += hog_pile(score1, score0)
             print(f's0 = {score0}, s1 = {score1}')
        else:
            score0 += take_turn(strategy0(score0, score1), score0, score1, dice, goal)
            score0 += hog_pile(score0, score1)
            print(f's0 = {score0}, s1 = {score1}')
        who = next_player(who)
    # END PROBLEM 5
    # (note that the indentation for the problem 7 prompt (***YOUR CODE HERE***) might be misleading)
    # BEGIN PROBLEM 7
    "*** YOUR CODE HERE ***"
    # END PROBLEM 7
    return score0, score1

In [21]:
def always_roll(n):
    """Return a strategy that always rolls N dice.

    A strategy is a function that takes two total scores as arguments (the
    current player's score, and the opponent's score), and returns a number of
    dice that the current player will roll this turn.

    >>> strategy = always_roll(5)
    >>> strategy(0, 0)
    5
    >>> strategy(99, 99)
    5
    """

    def strategy(score, opponent_score):
        return n

    return strategy

In [62]:
always = always_roll(1)
always_three = make_test_dice(2)
play(always, always, score0=0, score1=0, dice=always_three, goal=50)

s0 = 2, s1 = 0
s0 = 2, s1 = 4
s0 = 8, s1 = 4
s0 = 8, s1 = 6
s0 = 10, s1 = 6
s0 = 10, s1 = 8
s0 = 12, s1 = 8
s0 = 12, s1 = 10
s0 = 14, s1 = 10
s0 = 14, s1 = 12
s0 = 16, s1 = 12
s0 = 16, s1 = 14
s0 = 18, s1 = 14
s0 = 18, s1 = 16
s0 = 20, s1 = 16
s0 = 20, s1 = 18
s0 = 22, s1 = 18
s0 = 22, s1 = 20
s0 = 24, s1 = 20
s0 = 24, s1 = 22
s0 = 26, s1 = 22
s0 = 26, s1 = 24
s0 = 28, s1 = 24
s0 = 28, s1 = 26
s0 = 30, s1 = 26
s0 = 30, s1 = 28
s0 = 32, s1 = 28
s0 = 32, s1 = 30
s0 = 34, s1 = 30
s0 = 34, s1 = 32
s0 = 36, s1 = 32
s0 = 36, s1 = 34
s0 = 38, s1 = 34
s0 = 38, s1 = 36
s0 = 40, s1 = 36
s0 = 40, s1 = 38
s0 = 42, s1 = 38
s0 = 42, s1 = 40
s0 = 44, s1 = 40
s0 = 44, s1 = 42
s0 = 46, s1 = 42
s0 = 46, s1 = 44
s0 = 48, s1 = 44
s0 = 48, s1 = 46
s0 = 50, s1 = 46


(50, 46)

In [59]:
hefty_hogs(4, 12)

24

In [64]:
i = 1
a = 1

i, a = 0, 0
print(f'i = {i}, a = {a}')

i = 0, a = 0


In [82]:
def hefty_hogs_strategy(score, opponent_score, threshold=8, num_rolls=6):
    """This strategy returns 0 dice if that gives at least THRESHOLD points, and
    returns NUM_ROLLS otherwise.
    """
    # BEGIN PROBLEM 10
    if hefty_hogs(score, opponent_score) < threshold:
        return num_rolls
    else:
        return 0

In [90]:
hefty_hogs_strategy(64, 35, 12, 3)

3

In [80]:
hefty_hogs(20, 32)

15

In [91]:
def hog_pile_strategy(score, opponent_score, threshold=8, num_rolls=6):
    """This strategy returns 0 dice when this would result in Hog Pile taking
    effect. It also returns 0 dice if it gives at least THRESHOLD points.
    Otherwise, it returns NUM_ROLLS.
    """
    # BEGIN PROBLEM 11
    if hefty_hogs_strategy(score, opponent_score, threshold, num_rolls) == 0 or hog_pile(score + hefty_hogs(score, opponent_score), opponent_score):
        return 0
    else:
        return num_rolls

In [92]:
hog_pile_strategy(64, 35, 12, 3)

0

In [93]:
from operator import add

add(1, 2)

3